In [ ]:
# === Ignorar advertencias ===
import warnings
warnings.filterwarnings('ignore')

# === Manejo del sistema y utilidades ===
import os
import ast
from itertools import product
from tqdm import tqdm
from tqdm import trange  # Asegúrate de tenerlo importado
import time
from IPython.display import clear_output


# === Manipulación de datos ===
import numpy as np
import pandas as pd

# === Visualización ===
import matplotlib.pyplot as plt

# === Preprocesamiento y métricas ===
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score
)

# === Diagnóstico estadístico ===
from statsmodels.stats.diagnostic import acorr_ljungbox

# === Redes neuronales (Keras / TensorFlow) ===
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2



In [ ]:
# Listar dispositivos disponibles
devices = tf.config.list_physical_devices('GPU')

if devices:
    print(f"TensorFlow está utilizando la GPU: {devices}")
else:
    print("TensorFlow no está utilizando la GPU")

# Regresión

## Lectura de datos

In [ ]:
df = pd.read_csv('./datasets/data_treino_dv_df_2000_2010.csv')
df.head(1)

In [ ]:
df.columns = ['HORA','WIND_DIR_HOR','WIND_VEL_HOR','HUM_REL_MAX_ANT','HUM_REL_MIN_ANT','TEMP_MAX_ANT','TEMP_MIN_ANT','HUM_REL_HOR','PRES_ATM_NIV','PREC_HOR','RAFAGA_VIENTO','PRES_ATM_MAX_ANT','PRES_ATM_MIN_ANT']
df.head(2)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.drop(columns='HORA', inplace= True)

## Modelos con escalamiento de variables

Los siguientes modelos requieren escalamiento de las variables predictoras, por lo que se trabajan en la misma función

* MLP
* RNN
* LSTM

En un libro aparte, se realizó una exploración manual de hiperparámetros para cada uno de los modelos, y se estableció de forma experimental que los siguientes hiperparámetros son los que mejor se ajustan a cada uno de los modelos. En este libro, solo se modificará en el GridSearch uno o máximo dos (2) parámetros adicionales, para verificar si el modelo se ajusta mejor a los datos. 

* MLP: 2 capas ocultas laprimera de 64 neurnonas y la segunda de 32, activación tanh, optimizador Adam. Se probaran valores de learning rate y regularización L2.

In [ ]:
# 🧠 Construye un modelo MLP con capas ocultas personalizadas
def build_mlp_model(input_dim, hidden_layers, activation='relu', learning_rate=0.001, kernel_regularizer=None):
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_dim=input_dim, activation=activation, kernel_regularizer=kernel_regularizer))
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation, kernel_regularizer=kernel_regularizer))
    model.add(Dense(1))  # Capa de salida (regresión)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# 🧠 Construye un modelo RNN simple
def build_rnn_model(timesteps, input_dim, hidden_units, activation='tanh', learning_rate=0.001):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, activation=activation, input_shape=(timesteps, input_dim)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# 🧠 Construye un modelo LSTM para dependencias temporales largas
def build_lstm_model(timesteps, input_dim, hidden_units, activation='tanh', learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(hidden_units, activation=activation, input_shape=(timesteps, input_dim)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# 🧠 Genera secuencias de entrenamiento para RNN y LSTM
def create_rnn_sequences(df, target_col, window_size):
    X, y = [], []
    for i in range(len(df) - window_size):
        seq = df.iloc[i:i+window_size]
        X.append(seq.drop(columns=[target_col]).values)
        y.append(df.iloc[i+window_size][target_col])
    return np.array(X), np.array(y)

# 🔁 Función principal de sliding window para evaluación de modelos
def sliding_window_regression_models_scaling_keras_rnn(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7, 14],
    test_window=1,
    model_types=['MLP'],
    mlp_params={
        'hidden_layers': [[64, 32]],
        'activation': ['tanh'],
        'learning_rate': [0.001, 0.01],
        'epochs': [50],
        'batch_size': [32],
        'kernel_regularizer': [l2(0.001), l2(0.01)]
    },
    rnn_params={
        'hidden_units': [16, 32],
        'activation': ['tanh'],
        'learning_rate': [0.001],
        'epochs': [50],
        'batch_size': [32]
    },
    lstm_params={
        'hidden_units': [32, 64],
        'activation': ['tanh'],
        'learning_rate': [0.001],
        'epochs': [50],
        'batch_size': [32]
    },
    save_path='./progreso_keras_rnn'
):
    os.makedirs(save_path, exist_ok=True)
    resultados_por_T = {}

    log_tiempos_path = os.path.join(save_path, 'tiempos_ejecucion.csv')
    if not os.path.exists(log_tiempos_path):
        with open(log_tiempos_path, 'w') as f:
            f.write('T_dias,modelo,parametros,duracion_segundos\n')

    for T in tqdm(T_values, desc="Procesando ventanas T"):
        print(f"\n🧭 Iniciando evaluación para ventana T={T} días...")
        inicio_ventana = time.time()
        T_hours = T * 24
        test_hours = test_window * 24
        total_windows = len(df) - T_hours - test_hours + 1
        output_path = os.path.join(save_path, f'ANN_resultados_T{T}.csv')

        # Cargar resultados previos si existen
        if os.path.exists(output_path):
            df_prev = pd.read_csv(output_path)
            if 'params' in df_prev:
                df_prev['params'] = df_prev['params'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        else:
            df_prev = pd.DataFrame(columns=['modelo', 'params', 'T_dias', 'T_horas', 'MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p'])

        for model_type in model_types:
            if model_type == 'MLP':
                param_grid = list(product(*mlp_params.values()))
                param_keys = list(mlp_params.keys())
            elif model_type == 'RNN':
                param_grid = list(product(*rnn_params.values()))
                param_keys = list(rnn_params.keys())
            elif model_type == 'LSTM':
                param_grid = list(product(*lstm_params.values()))
                param_keys = list(lstm_params.keys())

            for combo in param_grid:
                param_dict = dict(zip(param_keys, combo))
                ya_realizado = not df_prev.empty and ((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict))).any()
                if ya_realizado:
                    rmse = df_prev[((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict)))]['RMSE'].values[0]
                    print(f"⏩ Saltando modelo ya evaluado: {model_type} con hiperparámetros: {param_dict} y RMSE {rmse}")
                    continue

                modelo_idx = param_grid.index(combo) + 1
                total_modelos = len(param_grid)
                progreso_modelo = (modelo_idx / total_modelos) * 100
                print(f"\n🔧 Modelo {modelo_idx}/{total_modelos} ({progreso_modelo:.1f}%) - {model_type} con hiperparámetros: {param_dict}")

                inicio_modelo = time.time()
                resultados = {k: [] for k in ['MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p']}

                for start in tqdm(range(0, total_windows, 24), desc=f"     ↪ Subventanas ({model_type})", leave=False):
                    if model_type == 'MLP':
                        train = df.iloc[start: start + T_hours]
                        test = df.iloc[start + T_hours: start + T_hours + test_hours]

                        X_train = train.drop(columns=[target_col])
                        y_train = train[target_col]
                        X_test = test.drop(columns=[target_col])
                        y_test = test[target_col]

                        scaler = StandardScaler()
                        X_train = scaler.fit_transform(X_train)
                        X_test = scaler.fit_transform(X_test)

                        model = build_mlp_model(
                            input_dim=X_train.shape[1],
                            hidden_layers=param_dict['hidden_layers'],
                            activation=param_dict['activation'],
                            learning_rate=param_dict['learning_rate'],
                            kernel_regularizer=param_dict['kernel_regularizer']
                        )

                    else:
                        # Para RNN o LSTM
                        continue
                        data_window = df.iloc[start: start + T_hours + test_hours].copy()
                        train_data = data_window.iloc[:T_hours]
                        test_data = data_window.iloc[T_hours:]

                        scaler = StandardScaler()
                        X_train_scaled = scaler.fit_transform(train_data.drop(columns=[target_col]))
                        X_test_scaled = scaler.fit_transform(test_data.drop(columns=[target_col]))

                        train_scaled = pd.DataFrame(X_train_scaled, columns=train_data.columns.drop(target_col))
                        train_scaled[target_col] = train_data[target_col].values
                        test_scaled = pd.DataFrame(X_test_scaled, columns=test_data.columns.drop(target_col))
                        test_scaled[target_col] = test_data[target_col].values
                        scaled = pd.concat([train_scaled, test_scaled])

                        X, y = create_rnn_sequences(scaled, target_col, T_hours)
                        X_train, y_train = X[:-1], y[:-1]
                        X_test, y_test = X[-1:], y[-1:]

                        if model_type == 'RNN':
                            model = build_rnn_model(X_train.shape[1], X_train.shape[2], param_dict['hidden_units'], param_dict['activation'], param_dict['learning_rate'])
                        else:
                            model = build_lstm_model(X_train.shape[1], X_train.shape[2], param_dict['hidden_units'], param_dict['activation'], param_dict['learning_rate'])

                    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
                    model.fit(X_train, y_train, epochs=param_dict['epochs'], batch_size=param_dict['batch_size'], verbose=0, callbacks=[early_stop])
                    y_pred = model.predict(X_test).flatten()

                    # Cálculo de métricas
                    residuals = y_test.values - y_pred if hasattr(y_test, 'values') else y_test - y_pred
                    resultados['MAPE'].append(mean_absolute_percentage_error(y_test, y_pred))
                    resultados['MAE'].append(mean_absolute_error(y_test, y_pred))
                    resultados['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
                    resultados['MSE'].append(mean_squared_error(y_test, y_pred))
                    resultados['R2'].append(r2_score(y_test, y_pred))
                    ljung_p = acorr_ljungbox(residuals, lags=[1], return_df=True)['lb_pvalue'].iloc[0] if len(residuals) >= 2 else np.nan
                    resultados['LjungBox_p'].append(ljung_p)

                # Guardar resultados de este modelo
                nuevo_row = pd.DataFrame([{
                    'modelo': model_type,
                    'params': param_dict,
                    'T_dias': T,
                    'T_horas': T_hours,
                    'MAPE': np.mean(resultados['MAPE']),
                    'MAE': np.mean(resultados['MAE']),
                    'RMSE': np.mean(resultados['RMSE']),
                    'MSE': np.mean(resultados['MSE']),
                    'R2': np.mean(resultados['R2']),
                    'LjungBox_p': np.nanmean(resultados['LjungBox_p'])
                }])
                df_prev = pd.concat([df_prev, nuevo_row], ignore_index=True)
                df_prev.to_csv(output_path, index=False)

                # Guardar duración
                fin_modelo = time.time()
                duracion = fin_modelo - inicio_modelo
                with open(log_tiempos_path, 'a') as f:
                    f.write(f'{T},{model_type},"{param_dict}",{duracion:.2f}\n')

                print(f"✅ Finalizado {model_type} con RMSE promedio: {np.mean(resultados['RMSE']):.4f} en {duracion:.2f} segundos")

        resultados_por_T[T] = df_prev
        duracion_ventana = time.time() - inicio_ventana
        print(f"🕒 Tiempo total para T={T} días: {duracion_ventana:.2f} segundos")

    return resultados_por_T


In [ ]:
resultados_por_T = sliding_window_regression_models_scaling_keras_rnn(df, target_col='WIND_VEL_HOR')

# Ver los DataFrames en memoria
df_w_7 = resultados_por_T[7]    # Para T=7 días
df_w_14 = resultados_por_T[14]    # Para T=7 días
df_w_21 = resultados_por_T[21]    # Para T=7 días


In [ ]:
# Ordenar por RMSE de mayor a menor
df_w_7_sorted = df_w_7.sort_values(by="RMSE", ascending=True)

# Etiquetas combinadas modelo + params
etiquetas = df_w_7_sorted['modelo'].astype(str) + ' ' + df_w_7_sorted['params'].astype(str)

# Crear la gráfica
plt.figure(figsize=(12, 6))
plt.bar(etiquetas, df_w_7_sorted['RMSE'])
plt.xticks(rotation=45, ha='right')
plt.ylabel('RMSE')
plt.title('RMSE por Modelo - T=7 días (ordenado de mayor a menor)')
plt.tight_layout()
plt.show()

## KNeighboors